In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
url = "https://raw.githubusercontent.com/CADETEMMA/Hematology_AI_Project/refs/heads/main/aml_tcga_gdc_clinical_data.csv"

df = pd.read_csv(url, sep="\t")

df.head()


,Study ID,Patient ID,Sample ID,Diagnosis Age,Biopsy Site,Cancer Type,Cancer Type Detailed,Last Communication Contact from Initial Pathologic Diagnosis Date,Birth from Initial Pathologic Diagnosis Date,Death from Initial Pathologic Diagnosis Date,...,Project Name,Project State,Race Category,Number of Samples Per Patient,Sample Type,Sample type id,Sex,TMB (nonsynonymous),Patient's Vital Status,Year of Diagnosis
0,aml_tcga_gdc,TCGA-AB-2802,TCGA-AB-2802-03,50,Bone Marrow,Acute Myeloid Leukemia,Acute Myeloid Leukemia,NaN,-18385,365.0,...,Acute Myeloid Leukemia,released,WHITE,1,Primary Blood Derived Cancer - Peripheral Blood,3,Male,NaN,Dead,2001
1,aml_tcga_gdc,TCGA-AB-2803,TCGA-AB-2803-03,61,Bone Marrow,Acute Myeloid Leukemia,Acute Myeloid Leukemia,NaN,-22584,792.0,...,Acute Myeloid Leukemia,released,WHITE,1,Primary Blood Derived Cancer - Peripheral Blood,3,Female,NaN,Dead,2001
2,aml_tcga_gdc,TCGA-AB-2804,TCGA-AB-2804-03,30,Bone Marrow,Acute Myeloid Leukemia,Acute Myeloid Leukemia,2557.0,-11203,NaN,...,Acute Myeloid Leukemia,released,WHITE,1,Primary Blood Derived Cancer - Peripheral Blood,3,Male,NaN,Alive,2001
3,aml_tcga_gdc,TCGA-AB-2805,TCGA-AB-2805-03,76,Bone Marrow,Acute Myeloid Leukemia,Acute Myeloid Leukemia,NaN,-28124,577.0,...,Acute Myeloid Leukemia,released,WHITE,1,Primary Blood Derived Cancer - Peripheral Blood,3,Male,NaN,Dead,2002
4,aml_tcga_gdc,TCGA-AB-2806,TCGA-AB-2806-03,46,Bone Marrow,Acute Myeloid Leukemia,Acute Myeloid Leukemia,NaN,-16892,945.0,...,Acute Myeloid Leukemia,released,WHITE,1,Primary Blood Derived Cancer - Peripheral Blood,3,Male,NaN,Dead,2002


In [ ]:
df.shape


(200, 33)

In [ ]:
df["Overall Survival Status"].value_counts()


,count
Overall Survival Status,
1:DECEASED,133
0:LIVING,67


In [ ]:
# Définission de la variable cible (transformation en variable binaire)
df["target"] = df["Overall Survival Status"].map({
    "1:DECEASED": 1,
    "0:LIVING": 0
})


df["target"].value_counts(normalize=True)



,proportion
target,
1,0.665
0,0.335


Choix des colonnes qui serviront à prédire la variable cible.
La variable “Overall Survival (Months)” a été exclue afin d’éviter toute fuite de données, car elle n’est pas disponible au moment de la prédiction.

In [ ]:
# Sélection des variables numériques (=sélection des features)
num_features = [
    "Diagnosis Age",
    "Mutation Count",

]


In [ ]:
# Sélection des variables catégorielles (=sélection des features)
cat_features = [
    "Sex",
    "Race Category",
    "Prior Treatment"
]


Remplace les valeurs manquantes NaN par la médiane ou la valeur la plus présente
Pipelines de preprocessing
OneHot -> compatible avec les modèles machine learning
L'encodage concerne les variables catégorielles car on doit les trasnformer en nombre.
Encodage = transformation en nb
Scaler agit sur les variables numériques, pour les mettre sur la même échelle

In [ ]:
# Pipeline pour variables numériques
numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Pipeline pour variables catégorielles
categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(
        handle_unknown="ignore",
        sparse_output=False
    ))
])

# Combinaison des pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, num_features),
        ("cat", categorical_pipeline, cat_features)
    ]
)


Les variables catégorielles sont encodées par one-hot encoding, ce qui augmente le nombre de features finales utilisées par le modèle. J’utilise get_feature_names_out() pour tracer précisément les variables réellement exploitées.

Les variables numériques → restent 1 colonne chacune

Les variables catégorielles → sont transformées en plusieurs colonnes
(une par modalité, via OneHotEncoder)

In [ ]:
# colonnes final après encodage
feature_names = preprocessor.get_feature_names_out()

print("Colonnes finales utilisées par le modèle :")
for col in feature_names:
    print("-", col)

print("\nNombre total de variables après encodage :", len(feature_names))


Colonnes finales utilisées par le modèle :
- num__Diagnosis Age
- num__Mutation Count
- num__Overall Survival (Months)
- cat__Sex_Female
- cat__Sex_Male
- cat__Race Category_ASIAN
- cat__Race Category_BLACK OR AFRICAN AMERICAN
- cat__Race Category_WHITE
- cat__Prior Treatment_False
- cat__Prior Treatment_True

Nombre total de variables après encodage : 10


In [ ]:
# séparation train/test
X = df[num_features + cat_features]
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)



Dimensions des matrices après le split :
- (160, 10) → X_train
160 lignes → 160 patients pour l’entraînement
10 colonnes → 10 variables explicatives (features)

après sélection
après encodage + scaling via le pipeline

- (40, 10) → X_test

40 lignes → 40 patients pour le test

10 colonnes → exactement les mêmes variables, même ordre


Conclusion
80 % entraînement
20 % test
pas de fuite de données
dimensions cohérentes
meme nb de colonnes train/test



In [ ]:
# application du préprocessing
# Fit uniquement sur le train
X_train_processed = preprocessor.fit_transform(X_train)

# Transformation du test
X_test_processed = preprocessor.transform(X_test)

# Vérification des dimensions
print(X_train_processed.shape)
print(X_test_processed.shape)


(160, 10)
(40, 10)


Les données ont été prétraitées à l’aide de pipelines scikit-learn distincts pour les variables numériques et catégorielles. L’ensemble du preprocessing est intégré dans un ColumnTransformer, garantissant une transformation cohérente des données et la prévention du data leakage.